### Import packages

Runs in python version 3.8.10

In [ ]:
import pandas as pd
import numpy as np
import numpy.matlib
import pygenstability
from pygenstability import run, plotting
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import pickle
import seaborn as sns

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors

# read in the two embedding files
sgm = pd.read_csv('SGM.csv', index_col='Condition')
mca = pd.read_csv('MCA.csv', index_col='Condition')

### Define ckNN function

In [ ]:
def ckNNGraph(D, k):
    n = D.shape[0]
    np.fill_diagonal(D, 0)
    delta = 1
    dk = np.zeros(n)

    for i in range(n):
        tmp = np.sort(D[i,:]) 
        dk[i] = tmp[k]

    Dk = np.outer(dk, dk.conj().transpose())

    E = D**2 < delta**2 * Dk

    np.fill_diagonal(E, 0)
    Emst = minimum_spanning_tree(D).toarray()
    Emst = np.sign(Emst)
    Emst = np.maximum(Emst, Emst.transpose())
    E = np.maximum(E, Emst)

    return E

### Select which embedding to use
Options: Multiple Correspondence Analysis (mca) or Skipgram with multiple codes (sgm)

In [ ]:
embedding = sgm

In [ ]:
# Determine cosine similarity
cosine = pd.DataFrame(cosine_similarity(embedding), index = embedding.index, columns = embedding.index)
cosine = cosine.to_numpy()

### Normalise and sparsen using MST-CkNN


In [ ]:
# Set knn value (we tested here values of 5, 10, 15 and 20)
knn = 10

In [ ]:
# convert to array and calculate cosine distance
dist = 1-cosine

# apply max normalisation to distance matrix
dist_norm = dist / np.amax(dist)

# calculate normalised similarity matrix (1 minus distance matrix in range [0,1])
cos_norm = 1-dist_norm

# apply cKNN
mstknn = ckNNGraph(dist_norm, knn)

# hadamard multiplication with normalised similarity matrix
matA = np.multiply(mstknn, cos_norm)

### Run Markov Multiscale Community Detection

For details of the implementation, and hyperparameter settings, see:
https://github.com/barahona-research-group/PyGenStability

In [ ]:
resultsA = run(matA,
                  constructor = 'linearized', # default = linearized
                  min_scale=-1.0, # default = -2.0
                  max_scale=0.5, # default = 0.5
                  n_scale=40, # default = 20
                  n_NVI=40, # default = 20
                  n_tries=200, # default = 100
                  method = "leiden",
                  with_optimal_scales=False,
                  with_postprocessing=True,
                  result_file='MCD_results_cknn'+str(knn)+'.pkl',
                  n_workers=4, # default = 4
             )

### Identify optimal scales

In [ ]:
from pygenstability.optimal_scales import identify_optimal_scales
results  = identify_optimal_scales(resultsA,
                                                  kernel_size=5, # default = 3
                                                  window_size=6, # default = 5
                                                  max_nvi=1, # default = 1
                                                  basin_radius=1 # default = 1
                                                 )

# print number of partitions
for x in results["selected_partitions"]:
    print(len(np.unique(results['community_id'][x])))

In [ ]:
#This returns an interactive and pdf plot of the output
plotting.plot_scan(results, use_plotly=True, live=False)

# plot results
plt.figure(figsize=(7, 6))
axes = plotting.plot_scan(results, figure_name=None)
axes[4].set(yticklabels=[0,0.05,0.10])

plt.savefig('PLOT_MCD_KNN_'+str(knn)+'.pdf')
plt.show()

# save out results as pickled list
with open('MCD_results_optimal_cknn'+str(knn)+'.pkl', "wb") as fp:
    pickle.dump(results, fp)